In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/master_degree/paper')
!ls

Mounted at /content/drive
'논문계획서 (2021.01.22).gdoc'		 '기존 분석결과.gdoc'
 archive.zip				 '논문 지도교수 배정신청서.gdoc'
 booking.com_hotel_review_korea.csv.zip  '레퍼런스 링크.gdoc'
 code_R					 '논문계획 정보모음.gdoc'
 colab					 '연구계획 표.gsheet'
 data					  model


In [87]:
import pandas as pd
import numpy as np
import json
import pickle
from io import StringIO
import sys

In [4]:
file_ls = os.listdir('data/downloads')

col_nm_init = ['id', 'review', 'rating', 'start_dt', 'dt', 'app_ver_long', 'app_ver', 'lang', 'lang_nm']
col_nm = ['app_nm', 'review', 'rating', 'dt', 'app_ver', 'lang']

df_raw = pd.DataFrame(columns = col_nm)

for f in file_ls:
  d = pd.read_csv('data/downloads/'+f, sep=',', skiprows = lambda x: x in [0,1,2,3,4,5,6,7,8], error_bad_lines=False)
  d.drop(d.tail(3).index, inplace=True)
  d.columns = col_nm_init
  d['app_nm'] = f.split('_')[1]
  d = d[col_nm]
  df_raw = df_raw.append(d)

In [27]:
df = df_raw[df_raw['lang']=='en']
df_ko = df_raw[df_raw['lang']=='ko']

df = df[df['app_nm'].isin(['zoom','fbm','discord'])]
df = df[df['review'].notnull()]
print(df.shape)
df

(1313952, 6)


,app_nm,review,rating,dt,app_ver,lang
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en
...,...,...,...,...,...,...
18673,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en
18674,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en
18675,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en
18676,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en


In [28]:
!pip install fasttext

import fasttext
!wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
path_to_pretrained_model = '/tmp/lid.176.bin'

text2 = 'In 1793, Alexander Hamilton recruited Webster to move to New York City and become an editor for a Federalist Party newspaper.'
fmodel = fasttext.load_model(path_to_pretrained_model)
print(fmodel.predict([text2]))  # ([['__label__en']], [array([0.9331119], dtype=float32)]

# filter out non-english
df['lang_ft'] = df['review'].apply(lambda x: fmodel.predict(x))
df['lang_en'] = df['lang_ft'].apply(lambda x: '__label__en' in list(x)[0])
df = df[df['lang_en']]
df.reset_index(inplace=True, drop=True)
print(df.shape)
df

     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3039063 sha256=cab3a66af9444f7e293463dcf7d7eb4c15599010314e3f3a59f518910a78340f
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
--2021-01-19 13:40:35--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/tmp/lid.176.bin’

/tmp/lid.176.bin    100%[===================>] 125.18M  68.1MB/s    in 1.8s    

2021-01-19 13:40:37 (68.1 MB/s) - ‘/tmp/lid.176.bin’ saved [131266198/131266198]

([['__label__en']], [array([0.9331119], dtype=float32)

(1140110, 8)


,app_nm,review,rating,dt,app_ver,lang,lang_ft,lang_en
0,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"((__label__en,), [0.9754365682601929])",True
1,zoom,Good,5.0,2020-12-26,5.4.6.812,en,"((__label__en,), [0.5064521431922913])",True
2,zoom,Good,4.0,2020-12-28,5.4.7.946,en,"((__label__en,), [0.5064521431922913])",True
3,zoom,Pro,5.0,2020-12-26,5.2.42588.0803,en,"((__label__en,), [0.25253161787986755])",True
4,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,en,"((__label__en,), [0.9959578514099121])",True
...,...,...,...,...,...,...,...,...
1140105,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en,"((__label__en,), [0.9687220454216003])",True
1140106,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en,"((__label__en,), [0.9431992173194885])",True
1140107,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en,"((__label__en,), [0.912397563457489])",True
1140108,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en,"((__label__en,), [0.9451221823692322])",True


In [32]:
df = df[['app_nm','review','rating','dt','app_ver']]
print(df.groupby('app_nm').size())
print("\n",df.info())
df

app_nm
discord    165190
fbm        673606
zoom       301314
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140110 entries, 0 to 1140109
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   app_nm   1140110 non-null  object 
 1   review   1140110 non-null  object 
 2   rating   1140110 non-null  float64
 3   dt       1140110 non-null  object 
 4   app_ver  1140110 non-null  object 
dtypes: float64(1), object(4)
memory usage: 43.5+ MB

 None


,app_nm,review,rating,dt,app_ver
0,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown
1,zoom,Good,5.0,2020-12-26,5.4.6.812
2,zoom,Good,4.0,2020-12-28,5.4.7.946
3,zoom,Pro,5.0,2020-12-26,5.2.42588.0803
4,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946
...,...,...,...,...,...
1140105,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119
1140106,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129
1140107,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129
1140108,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129


### Preprocessing
- tokenize
- lemmatize
- stop words
- detokenize
- only contains English

### Columns
- app name
- review
- rating
- tokens_l (list)
- tokens_s (set)
- tokens_d (dict)
- detokens
- n_words (from list)
- n_words_unq (from set)


In [50]:
rv = df['review']
rv = rv.tolist()
rv[:20]

['This is our one of the education thing. We cant unlike this never',
 'Good',
 'Good',
 'Pro',
 "It's very very useful for school students",
 'Max 4 people on one screen including myself. This is not usable',
 'Super',
 "Won't let me Unmute. . The level of Simplicity is too complicated.",
 'Voice bug and video clarity not good',
 'Sucks',
 'Not working share option',
 'Very good',
 "It's amazing",
 'Woooooow',
 'Super it works',
 "Most of the time I can't join. After accepting my joining request by the host the screen shows blank or connecting. I've missed many of my class due to this problem...",
 'Aaje land mera',
 'Nice',
 'Worst app I dont want online classes',
 'Awesome']

In [97]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
import re

nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [70]:
rv_new = []
for l in rv:
  for elem in sent_tokenize(l):
    rv_new.append(elem)

NameError: ignored

In [88]:
rv_new = np.array(rv_new)
print(len(rv))
print(len(rv_new))

1140110
1432188


In [89]:
rv_new

array(['This is our one of the education thing.',
       'We cant unlike this never', 'Good', ..., '!',
       'Censor this Facebook!!', '!🖕🖕🖕🖕🖕'], dtype='<U862')

In [108]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def preprocess_text(sentence):
    tokeniser = RegexpTokenizer("[\s]+", gaps=True)
    lemmatizer = WordNetLemmatizer()

    tokens = " ".join(tokeniser.tokenize(sentence))
    tokens = re.sub('[=.#/?:!$}]', '', tokens)
    #tokenize the sentence and find the POS tag for each token
    # sentence2 = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', sentence)
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(tokens))  
    
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmas = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmas.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmas.append(lemmatizer.lemmatize(word, tag))
    detoken = " ".join(lemmas)
    keywords_l = [lemma for lemma in lemmas if lemma not in stopwords.words('english')] 
    keywords_s = list(set(keywords_l))
    freqs = dict((x, keywords_l.count(x)) for x in keywords_s)

    return lemmas, keywords_l, keywords_s, detoken, freqs

# def preprocess_text(lemmas):
#     keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')] 
#     return keywords

# def build_freqs(text):
#     return dict((x, text.count(x)) for x in set(text))

# >>> l = ["a","b","b"]
# >>> [[x,l.count(x)] for x in set(l)]
# [['a', 1], ['b', 2]]
# >>> dict((x,l.count(x)) for x in set(l))
# {'a': 1, 'b': 2}


In [110]:
# text = "Many existing state-of-the-art models are first pre-trained on a large text corpus and then fine-tuned on downstream tasks."
text = "Censor this Facebook!!"
lemmas, keywords_l, keywords_s, detoken, freqs = preprocess_text(text) 
# keywords = preprocess_text(lemm)
print(lemmas)
print(detoken)
print(keywords_l)
print(keywords_s)
print(freqs)

['Censor', 'this', 'Facebook']
Censor this Facebook
['Censor', 'Facebook']
['Facebook', 'Censor']
{'Facebook': 1, 'Censor': 1}


In [ ]:
# word 추출 (list)
keywords_l = []
for i in range(len(rv)):
  a = list(rv_dict[i].keys())
  keywords_l.append(a)

In [ ]:
# 역토큰화 (list)
detoken_1 = []
for i in range(len(rv)):
  a = ' '.join(keywords_l[i])
  detoken_1.append(a)

In [ ]:
# key value (list)
items_l = []
for i in range(len(rv)):
  a = rv_dict[i]
  items_l.append(a)

In [ ]:
df2 = pd.DataFrame.from_dict({'keywords':keywords_l, 
                              'detoken':detoken_1,
                              'items': items_l})

In [ ]:
df_m = pd.merge(df, df2, left_index=True, right_index=True)

In [ ]:
df_en.to_pickle('data/processed/rv_en_1.pkl')
df_ko.to_pickle('data/processed/rv_ko_1.pkl')

In [ ]:
import csv
import json

csvfile = open('file.csv', 'r')
jsonfile = open('file.json', 'w')

fieldnames = ("FirstName","LastName","IDNumber","Message")
reader = csv.DictReader( csvfile, fieldnames)
out = json.dumps( [ row for row in reader ] )
jsonfile.write(out)